In [ ]:
# This is Oliver's first attempt at sort of baseline model to predict colexification in CLICS. 
# Here, I was trying to predict p(a and b colexify) in a "randomly selected" language
# More specifically, I tried doing this by trying to predict the "colex.freq" column so I couuld divide these predictions by the number of languages in CLICS
# This is because if a and b colexify 300 times out of 5000 languages, 300/5000 is a good estimate for the probability a and b colexify in a "random" language
# To predict this, I created a model that recieved pairs of embeddings of senses as input
# I then viewed predicting p(a and b colexify) as a regression task, with the dependent variable being this probability as measured above

In [ ]:
from google.colab import drive
! [ -e /content ] && pip install -Uqq fastbook
! pip install torch-lr-finder
import fastbook
fastbook.setup_book()
from fastai.tabular.all import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastbook import *
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
import gensim.downloader as gs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random as r
from torch_lr_finder import LRFinder

In [ ]:
wv = gs.load('word2vec-google-news-300') # These are the word2vec embeddings we are using

In [ ]:
NUM_LANGUAGE = 3156 # This is the number of languages used in the CLICS database
df = pd.read_csv("gdrive/MyDrive/clics-colexification-data.csv") 
df.head()

,colex.freq,Concepticon_Gloss.xo,Concepticon_Gloss.yo,vision,assoc,affec,tax,fully_covered
0,340,TREE,WOOD,1,1,1,1,1
1,326,LEG,FOOT,1,1,1,1,1
2,296,MOON,MONTH,0,1,1,1,0
3,291,GO,WALK,0,1,1,1,0
4,284,HAND,ARM,1,1,1,1,1


In [ ]:
dumb_british_spellings = {"armour":"armor", "grey":"gray", "mould":"mold", "neighbour":"neighbor", "axe":"ax", "moustache":"mustache", "plough":"plow", "mandarine":"mandarin"}
obscure_words = {"shoulderblade":"shoulder blade", "spearthrower":"spear thrower", "ridgepole":"ridge pole", "pimpleface":"pimple face", "tumpline":"backpack", "cushma":"clothing", "curassow":"tropical bird", "banisterium":"plant", "paca":"rodent", "netbag":"net bag", "muntjacs":"barking deer"}

# This converts a sense to its word2vec embeddings. More specifically, for each sense I remove punctuation and then add the vectors of each individual word. 
# I also replace some obscure words and british spellings not recognized by word2vec to similar phrases that word2vec can recognize
def stringToVec(s):  
  s = s.lower()
  for i in "(),\t\n":
    s = s.replace(i, "")
  for i in "-":
    s = s.replace(i, " ")
  for i in dumb_british_spellings:
    s = s.replace(i, dumb_british_spellings[i])
  for i in obscure_words:
    s = s.replace(i, obscure_words[i])
  arr = s.split(" ")
  vec_defined = False
  for i in arr:
    try:      # try except skips over words in the sense that aren't in word2vec, e.g., "of"
      if vec_defined:
        vec += np.array(wv[i])
      else:
        vec = np.array(wv[i])
        vec_defined = True
    except:
      continue
  if not vec_defined:
    raise Exception("Word cannot be converted to vector") # raises an error if all the words in our sense weren't in word2vec
  return vec 
def toTensor(arr):# Function to easily convert arrays to tensors
  return torch.tensor(arr, dtype = torch.float32) 

In [ ]:
dep_var = "colex.freq" # Column name of dependent variable

dic = {} # This is a dictionary from which we create a new dataframe containing pairs of embeddings of senses rather than senses themselves
embed_length = len(wv["test"]) # Length of vectors in word2vec
for j in range(2*embed_length): # columns 0 - 599 are embeddings of the first sense, 600 - 1199 are embeddings of the second sense
  dic[j] = []
dic[dep_var] = []

vec_dic = {} # This will be a dictionary that easily allows us to access the embedding for all of our senses. Note this is not as simple as just calling wv[s] as some senses are phrases
sensePairs = set() # This is a set that keeps track of all the pairs of senses that appear in our data set, which will be used to add pairs of senses that don't appear
for i in range(len(df)):
  row = df.iloc[i]
  x = row["Concepticon_Gloss.xo"]
  y = row["Concepticon_Gloss.yo"]
  sensePairs.add(x+y)
  sensePairs.add(y+x)
  try:     # We use "try except" to pass over senses that give an error when trying to convert to word2vec embeddings
    if x in vec_dic:
      xvec = vec_dic[x]
    else:
      xvec = stringToVec(x)
      vec_dic[x] = xvec
    
    if y in vec_dic:
      yvec = vec_dic[x]
    else:
      yvec = stringToVec(y)
      vec_dic[y] = yvec
    
    for j in range(embed_length):
      dic[j].append(xvec[j])
    for j in range(embed_length):
      dic[embed_length+j].append(yvec[j])
    dic[dep_var].append(row[dep_var]) # Divides by NUM_LANGUAGE to convert colexification frequencies to probabilities
  except:
    continue

# To get more data, we now add pairs of senses that never colexify in our data set, giving probabilities 0
# The number of pairs we add it 10% of the size of our data set 

num_zeros = int(len(df)/10)
senses = list(vec_dic.keys())
sense_indices = {senses[i]:i for i in range(len(senses))}
for i in range(num_zeros):
  x = r.randint(0, len(senses)-1) # Here we take two random senses that never colexify
  y = r.randint(0, len(senses)-1)
  while senses[x]+senses[y] in sensePairs:
    x = r.randint(0, len(senses)-1)
    y = r.randint(0, len(senses)-1)
  sensePairs.add(senses[x]+senses[y])
  xvec = stringToVec(senses[x])
  yvec = stringToVec(senses[y])
  word_arr = np.concatenate((xvec, yvec))
  for j in range(len(word_arr)):
    dic[j].append(word_arr[j])
  dic[dep_var].append(0)

df_w2v = pd.DataFrame.from_dict(dic) # This is our new dataframe. We have replaced Conceptican_gloss.x, Conceptican_gloss.y with their embeddings
df_w2v.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,colex.freq
0,0.484375,0.122559,-0.157227,0.034668,-0.219727,-0.235352,0.113281,0.027710,0.132812,0.287109,0.105469,-0.241211,0.019897,0.033203,-0.069336,-0.082031,-0.259766,-0.187500,-0.006439,0.090332,0.007599,-0.076660,-0.104980,-0.125000,0.189453,-0.121582,-0.184570,0.047852,0.220703,-0.257812,-0.047607,-0.219727,-0.030273,-0.134766,-0.045410,-0.281250,-0.066406,-0.373047,0.027100,0.022461,0.150391,-0.146484,0.146484,-0.208008,0.128906,-0.240234,-0.294922,0.079590,0.025513,0.071777,-0.044189,0.115723,0.091797,-0.037598,0.279297,-0.063477,0.022827,0.147461,-0.028320,-0.077148,0.082031,-0.134766,0.209961,0.085449,-0.164062,-0.116699,0.166992,-0.093750,-0.111328,-0.002975,0.135742,-0.206055,-0.078125,-0.433594,0.049316,0.118652,0.210938,-0.351562,0.100586,-0.002914,0.097656,0.478516,-0.014099,-0.159180,-0.121582,0.035889,-0.204102,-0.027954,-0.044922,0.198242,-0.119629,0.133789,0.004730,-0.225586,-0.322266,-0.046875,0.036377,-0.310547,0.048828,-0.322266,-0.182617,-0.175781,0.248047,0.269531,0.214844,0.026123,0.100586,-0.109375,-0.388672,-0.312500,-0.597656,-0.322266,0.068359,-0.067871,0.125977,-0.222656,0.164062,0.024414,-0.251953,0.084961,0.046143,0.104492,-0.087402,0.172852,0.135742,0.229492,0.059082,-0.090332,0.042236,0.151367,-0.117676,-0.335938,0.017212,-0.369141,0.080566,0.384766,-0.210938,-0.135742,0.242188,0.082031,0.013794,0.031250,-0.113770,0.294922,0.157227,0.205078,0.028198,-0.227539,-0.171875,-0.036133,0.435547,0.044678,-0.247070,0.312500,-0.026367,-0.199219,0.008789,-0.085449,-0.036621,-0.112305,-0.179688,0.106445,0.170898,0.182617,-0.049805,-0.287109,0.127930,-0.087891,-0.343750,0.047119,-0.168945,0.223633,-0.241211,-0.083984,-0.324219,-0.137695,0.147461,0.200195,-0

In [ ]:
#This is a regression neural network in fastai 

procs_nn = [Categorify]  # Here are processes that fastai applies to the data. I've added categorify which helps deal with categorical data, but in this codes present state this won't do anything as we already dealt with this by creating the embeddings
cont_nn, cat_nn = cont_cat_split(df_w2v, dep_var=dep_var) # Splits data into continuous and categorical. Again, everything should be continuous
train_idx, test_idx = train_test_split(df_w2v.index, test_size=0.2, random_state = 0)
splits = (list(train_idx), list(test_idx))
to_nn = TabularPandas(df_w2v, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var)
dls = to_nn.dataloaders(1024) # 1024 is batch size 
colex_model_1 = tabular_learner(dls, y_range=(0,NUM_LANGUAGE), layers=[500, 500, 500, 100, 100, 100, 100],
                        n_out=1, loss_func=F.mse_loss)   # layers gives the sizes of linear layers we are using in our neural network

In [ ]:
colex_model_1.lr_find() # Finds learning rate

In [ ]:
colex_model_1.fit_one_cycle(100, 0.003) # Fits model for 100 epochs. fit_one_cycle is an improved variation of fit that includes things like momentum and cosine annealing. 

In [ ]:
# test_words allows us to predict p(a , b colexify) from our model for two words a, b

def words2tensor(word1, word2):
  return torch.tensor(np.concatenate((np.array(wv[word1]), np.array(wv[word2]), np.array([0, 0, 0, 0, 0]))), dtype = torch.float32)
def words2df(word1, word2):
  vec1 = np.array(wv[word1])
  vec2 = np.array(wv[word2])
  vec = np.concatenate((vec1, vec2))
  dic = {}
  for i in range(len(vec)):
    dic[i] = [vec[i]]
  return pd.DataFrame.from_dict(dic)
def test_words(word1, word2, model):
  dl = model.dls.test_dl(words2df(word1, word2))
  return model.get_preds(dl=dl)

print(test_words("flee", "trio", colex_net))

(tensor([[1505.2473]]), None)
